In [ ]:
!export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda-11.8/
!export CUDA_DIR="/usr/local/cuda-11.8/"
!export TF_GPU_ALLOCATOR=cuda_malloc_async

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras


gpus = tf.config.list_physical_devices("GPU")
print(f"gpus={gpus}")

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display # If using IPython, Colab or Jupyter
import numpy as np
import tensorflow_addons as tfa
import datetime
import random
import time

In [ ]:
mnist = tf.keras.datasets.fashion_mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
from utils import NoiseUtil, ImgUtils, DataLoader, DataManager
           
def train_get():
   for x, y in zip(x_train, y_train):
        x = tf.expand_dims(x, axis=2)
        yield x

def test_get():
   for x in x_test:
        x = tf.expand_dims(x, axis=2)
        yield x


def add_noise(x,y):
    n = NoiseUtil.pixel_noise(x, 25, 5)

#     n = x + 0.4 * tf.random.normal(
#         x.shape[1:],
#         mean=0.0,
#         stddev=1.0,
#         dtype=tf.dtypes.float32,
#     )

    return n,y
dm = DataManager.create_label_with_input_transform(train_get, test_get, (28,28,1), add_noise)
dm.print_validation() 

In [ ]:
# it = iter(dm.get_test_data(2))
# i,j = next(it)
# plt.imshow(ImgUtils.denormalize(j, cast=tf.uint8)[1])
# plt.figure()
# plt.imshow(ImgUtils.denormalize(NoiseUtil.pixel_noise(j, 10, 5), cast=tf.uint8)[1])

In [ ]:
# im = tf.reshape(tf.constant(range(2*12*12*1)), (2,12,12,1))

# NoiseUtil.pixel_noise(im, 10, 2)[:,:,:,0]

In [ ]:
print(x_test[0].shape)

In [ ]:
LATENT_SIZE = 32

In [ ]:
def get_conv_t(filters, norm=False):
    conv = tf.keras.Sequential()
    if norm:
        conv.add(tfa.layers.SpectralNormalization(
            layers.Conv2DTranspose(filters, 2, 2, padding="same")
        ))
    else:
        conv.add(layers.Conv2DTranspose(filters, 2, 2, padding="same"))
    conv.add(layers.LeakyReLU())
    return conv

class InceptionBlock(layers.Layer):

    def __init__(self, filters=32, strides=1):
        super(InceptionBlock, self).__init__()
        self.filters = filters
        self.strides = strides
    
    def build(self, input_shape):  # Create the state of the layer (weights)
        filters = self.filters
        self.dropout = layers.Dropout(0.5)
        self.conv_x1 = get_conv(filters, kernel=2, strides=self.strides)
        self.conv_x2 = get_conv(filters, kernel=3, strides=self.strides)
        self.conv_x3 = get_conv(filters, kernel=5, strides=self.strides)
        self.out = get_conv(filters, kernel=1, strides=1)

    def call(self, inputs):  # Defines the computation from inputs to outputs
        x1 = self.conv_x1(inputs)
        x2 = self.conv_x2(inputs)
        x3 = self.conv_x3(inputs)

        x = tf.concat([x1,x2,x3], axis=-1)
#         x = self.dropout(x)
        x = self.out(x)
        return x

def get_conv(filters, kernel=2, strides=2, norm=False):
    conv = tf.keras.Sequential()
    if norm:
        conv.add(tfa.layers.SpectralNormalization(
            layers.Conv2D(filters, kernel, strides, padding="same")
        ))
    else:
        conv.add(layers.Conv2D(filters, kernel, strides, padding="same"))
    conv.add(layers.LeakyReLU())
    return conv
    
def get_encoder():
    encoder = tf.keras.Sequential(name="encoder")
    # encoder.add(layers.GaussianDropout(0.2))
    encoder.add(InceptionBlock(32))
    encoder.add(layers.Dropout(0.5))
    
    encoder.add(get_conv(128, 3, 1, norm=True))
    encoder.add(layers.Dropout(0.5))
    
    encoder.add(get_conv(256, 3, 2, norm=True))
    encoder.add(layers.Dropout(0.5))

    encoder.add(get_conv(512, 3, 2, norm=True))
    encoder.add(layers.Dropout(0.5))
    return encoder

def get_decoder():
    decoder = tf.keras.Sequential(name="decoder")
    decoder.add(get_conv_t(256, norm=True))
    decoder.add(layers.Dropout(0.5))
    decoder.add(get_conv(128, 3, 1))
    decoder.add(layers.Dropout(0.5))
    decoder.add(get_conv_t(32))
    decoder.add(layers.Dropout(0.5))
    decoder.add(get_conv(3, 1, 1))
    decoder.add(layers.Dropout(0.5))
    decoder.add(layers.Conv2D(1, 3, padding='same', activation='sigmoid'))
    return decoder

class AutoEncoder(tf.keras.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    
  def build(self, input_shape):
    self.encoder = get_encoder()
    self.decoder = get_decoder()
    self.encoder.build(input_shape=input_shape)
    sh = self.encoder.output_shape
    self.flatten = layers.Flatten()
    self.seq1 = layers.Dense(1024)
    self.reshape = layers.Reshape([*sh[1:]])
    self.inputs_dropout = layers.Dropout(0.5)

    
  def call(self, inputs):
#     x = self.inputs_dropout(inputs)
    x = self.encoder(inputs)
    x = self.flatten(x)
    x = self.reshape(x)
    x = self.inputs_dropout(x)
    x = self.decoder(x)
    
    # x_inputs = self.conv_input(inputs)
    
    # x = self.add([x * self.gamma, x_inputs])
    # x = self.attention(x)
#     x = self.last(x)
    return x

In [ ]:
from utils import SSIM, SSIM_Multiscale

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'logs/minst/minst_v2_spectral4'
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = train_log_dir,
  write_graph=True,
  histogram_freq = 1,
  update_freq="batch"
  )

EPOCHS = 60
model = AutoEncoder()

class SkMetrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.validation_loss = []
        self.epoch_n = 0
        self.start = time.time()
        tf.summary.scalar("start_time", self.start )

    def on_train_end(self, logs={}):
        self.end = time.time()
        tf.summary.scalar("end_time", self.end )
        tf.summary.scalar("training_time", self.end-self.start)

    def on_epoch_begin(self, epoch, logs):
        self.epoch_start = time.time()
        tf.summary.scalar("epoch_start_time", self.epoch_start, step=epoch)

    def on_epoch_end(self, epoch, logs):
        self.epoch_n = epoch
        self.epoch_end = time.time()
        tf.summary.scalar("epoch_end_time", self.epoch_end, step=epoch)
        tf.summary.scalar("epoch_total_time", self.epoch_end-self.epoch_start, step=epoch)

    def on_batch_end(self, batch, logs={}):
        def expand_and_predict(x):
            return model(x, training=False)
        if batch % 499 == 0 and batch > 0:
            display.clear_output()
            dm.print_validation(expand_and_predict)
        
class CustomMSE(keras.losses.Loss):
    def __init__(self, name="custom_mse"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        return mse

BATCH_SIZE = 25
epochs=100
steps_per_epoch=500

boundaries = [steps_per_epoch*10, steps_per_epoch*25, steps_per_epoch*60, steps_per_epoch*90, steps_per_epoch*95]
values = [0.001, 0.00075, 0.0005, 0.0001, 0.00005, 0.00001]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

cross_entropy = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
# opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# model.compile(run_eagerly=True)
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=opt, metrics=["mse", "accuracy", SSIM()])
history = model.fit(dm.get_training_data(BATCH_SIZE).repeat(), epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=dm.get_test_data(BATCH_SIZE), callbacks=[SkMetrics(), tboard_callback])
